In [6]:
from astropy.modeling import models
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from astropy import units as u
from astropy import constants as c

## Problem 13.5 ###

### Given: ###
Star HD 209458 has the following properties:
* M_s = 1.1 M_\odot
* L_s = 1.6 L_\odot
* T_s = 6000 K

It is orbited by a planet with
* M_p = 0.69 M_{jup}
* R_p = 1.35 R_{jup}
* d = 0.045 AU


### Find: ####

Would hydrogen evaportate from the star?


### Approach: ###

* Solve for escape temperature and temperature of the planet.  If $T_{planet} > T_{esc}$, then gas will evaporate.
* Need to estimate the albedo - will use the albedo of gas giants, $a \approx 0.5$.

In [81]:
#star properties
M_s = 1.1 *c.M_sun
L_s = 1.6 *c.L_sun
T_s = 6000 *u.K

# planet properties
M_p = 0.69 * c.M_jup
R_p = 1.35 * c.R_jup
d = 0.045*c.au
a = 0.5


### Temperature of Planet ###

Use Eqn 13.15

$$ T_p = T_s (1-a)^{1/4} \left(\frac{R_s}{2D}\right)^{1/2} $$

Need to solve for radius of Sun using BB equation:
$$ L = 4 \pi R_s^2 \sigma T^4 $$

So $$R_s = \sqrt{\frac{L}{4 \pi \sigma T^4}}$$

In [87]:
R_s = np.sqrt(L_s/(4*np.pi*c.sigma_sb*T_s**4))
print(R_s)

814390950.0410734 m


In [90]:
# From textbook Eqn 13.15
T_p = (1-a)**(.25)*T_s*np.sqrt(R_s/2/d)
print(T_p)

1240.869929267471 K


### Escape Temperature ###

$$T_{esc} = \frac{1}{54} \frac{G M m}{k R} $$

* M is the mass of the planet
* m is the mass of the gas atom in question, molecular hydrogen or $H_2$ ($2 \times m_{proton}$)


In [94]:
m_H2 = 2*c.m_p
T_esc = 1./54*c.G*M_p* m_H2/(c.k_B*R_p)
print(T_esc)

4063.8643107803614 K kg m2 / (J s2)


### Conclusion ###

The planet is hot ($T = 1240$~K), but it is not hot enough to evaporate $H_2$, which has an escape temperature of $T = 4064$~K.

## Problem 13.6  - Detecting Light from Planets##


### Given:###

* set of wavelengths
* temp and luminosity of the star
* temp and radius of the planet

### Find: ###

* luminosity output of star vs planet as a function of wavelength

### Aproach: ###

* calculate the luminosity of the star and planet.
* Use astropy's blackbody1D function to calculate the flux output at each wavelength.  
* calculate the ratio of $L_{star}/L_{planet}$ as a function of wavelength.


In [76]:
wavelengths = np.array([450., 700.,2200.,10000.]) # wavelengths is nm

In [77]:
Tstar = 6000 * u.K
Lstar = 1.61*c.L_sun
bbstar = models.BlackBody1D(Tstar, Lstar)

In [78]:
Tplanet = 1500. * u.K
Rplanet = 1.35 * c.R_jup
Lplanet = c.sigma_sb * Tplanet**4 * 4 * np.pi * Rplanet**2
a = .5 #albedo of the planet
bbplanet = models.BlackBody1D(Tplanet, Lplanet)

In [79]:
for w in wavelengths:
    print('wavelength = ',w)
    s = bbstar(w*u.nm)
    p = bbplanet(w*u.nm)
    #print(s)
    #print(p)
    print('\t star        = ',s)
    print('\t planet      = ',p)
    print('\t star/planet = ',s.value/p.value)

wavelength =  450.0
	 star        =  559811050493.7655 W / Hz
	 planet      =  887.0107753381873 W / Hz
	 star/planet =  631120913.1369666
wavelength =  700.0
	 star        =  1026030373401.1783 W / Hz
	 planet      =  476841.09685278946 W / Hz
	 star/planet =  2151723.876513804
wavelength =  2200.0
	 star        =  497936507623.811 W / Hz
	 planet      =  177612054.96402866 W / Hz
	 star/planet =  2803.5062582022197
wavelength =  10000.0
	 star        =  38626446594.90183 W / Hz
	 planet      =  90769388.42491443 W / Hz
	 star/planet =  425.5448589570933


Conclusion:  The ratio of starlight to planet light is lowest in infrared wavelengths, so that is where you should try to image the planet.  In fact, astronomers who are trying to take direct images of planets usually work at 10 microns.

## Problem 13.8  - Eris ##